In [ ]:
import torch 
import matplotlib.pyplot as plt
import random
import torch.nn as nn

In [ ]:
# --------------Torchifying MLP yayy---------------

In [ ]:
# Similiar api to that of pytorch

In [ ]:
class Linear:                  # torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)

    def __init__ (self, fan_in, fan_out, bias=True):              # fan in-out tells the number of inputs and outputs
        self.weights = torch.randn((fan_in, fan_out), generator=g)  # Complete fan in out later
        self.bias = torch.zeros(fan_out) if bias else None
                                                               
    def __call__(self, X):
        if self.bias:
            self.h = X @ self.weights + self.bias
        else:
            self.h = X @ self.weights
        return self.h
    
    def parameters(self):
            return sum(self.weights, ([] if self.bias is None else [self.bias]))

class BatchNorm1d:
# BatchNorm1d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)

    def __init__(self, fan_in, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(fan_in)
        self.beta = torch.zeros(fan_in)
    def __call__(self, X, Y):
        mean = X.mean(0, keepdim=True)
        var = X.var(0, keepdim=True)
#         y = (x - mean) / sqrt var[x] + epsilon  * gamma + beta
        self.Xhat = (X - mean) / torch.sqrt(var + self.eps) * self.gamma + self.beta # Xhat will represent the normalized version of the input
    
    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []

g = torch.Generator().manual_seed(2147483647)               
n_emb = 10                                     # Will represent the dim
n_hidden = 100                                 # Represents the hidden layer neurons
vocab_size = 27
block_size = 3

C = torch.randn((vocab_size, n_emb),            generator=g)
layers = [
      Linear(n_emb * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
      Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
      Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
      Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
      Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
      Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]

In [ ]:
losses = []
max_steps = 200000
batch_size = 32

for i in range(max_steps):
    
    # minibatch construct
    ix = torch.randint(0, X.shape[0], (batch_size,), generator=g)
    Xb, Yb = X[ix], Y[ix] # Batch of X & Y
    
    # Forward pass
    emb = C[Xb] # embed the characters into vectors
    x = emb.view(emb.shape[0], -1) # concatenate the vectors
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb) # loss function
    
#     Backward pass
    for layer in layers:
        layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        if p.grad == None:
            continue
        p.data += -lr * p.grad

    # track stats
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    losses.append(loss.log10().item())